## Exploring metadata for dashboard

In [61]:
from backend.connect_to_api import ResRobot

resrobot = ResRobot()

resrobot.access_id_from_location("Sandviken")

Name                                               extId
Sandviken station                                  740000195
Sandviken Agavägen                                 740019663
Sandviken Älgstigen                                740019983
Sandviken Bessemergymnasiet                        740019709
Sandviken Björksätra mitt                          740019711
Sandviken Bruket                                   740019718
Sandviken centrum                                  740019722
Sandviken Djurkliniken                             740019730
Sandviken Enen                                     740019732
Sandviken Göransson Arena                          740012166


In [62]:
resrobot.access_id_from_location("Göteborg")

Name                                               extId
GÖTEBORG                                           740098001
Göteborg Centralstation                            740000002
Göteborg Sävenäs lokstation                        740016365
GÖTEBORG GAMLESTADEN                               740098526
Göteborg Korsvägen                                 740015578
Göteborg Kungsportsplatsen                         740016358
Göteborg Stenpiren                                 740072430
Göteborg Eketrägatan                               740025624
Göteborg Vårväderstorget                           740025707
Göteborg Axel Dahlströms torg                      740025608


In [63]:
from dotenv import load_dotenv
import os 
import requests

load_dotenv()

API_KEY = os.getenv("API_KEY")

url = f"https://api.resrobot.se/v2.1/trip?format=json&originId=740000195&destId=740000002&passlist=true&showPassingPoints=true&accessId={API_KEY}"

response = requests.get(url)

result = response.json()
result.keys()

dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])

In [64]:
example_trip = result["Trip"][0]
example_trip.keys()

dict_keys(['Origin', 'Destination', 'ServiceDays', 'LegList', 'calculation', 'TripStatus', 'idx', 'tripId', 'ctxRecon', 'duration', 'rtDuration', 'checksum', 'transferCount'])

In [65]:
example_trip["LegList"]["Leg"][0].keys()

dict_keys(['Origin', 'Destination', 'Notes', 'JourneyDetailRef', 'JourneyStatus', 'Product', 'Stops', 'JourneyDetail', 'id', 'idx', 'name', 'number', 'category', 'type', 'reachable', 'waitingState', 'direction', 'directionFlag', 'duration', 'minimumChangeDuration'])

In [66]:
stops = example_trip["LegList"]["Leg"][0]["Stops"]["Stop"]
stops

[{'name': 'Sandviken station',
  'id': 'A=1@O=Sandviken station@X=16777881@Y=60614344@U=1@L=740000195@',
  'extId': '740000195',
  'routeIdx': 4,
  'lon': 16.777881,
  'lat': 60.614344,
  'depTime': '10:19:00',
  'depDate': '2025-01-22',
  'depDir': 'Mumman (Gävle kn)',
  'minimumChangeDuration': 'PT10M'},
 {'name': 'Sandviken Norra Porten',
  'id': 'A=1@O=Sandviken Norra Porten@X=16789584@Y=60619827@U=1@L=740019984@',
  'extId': '740019984',
  'routeIdx': 5,
  'lon': 16.789584,
  'lat': 60.619827,
  'depTime': '10:20:00',
  'depDate': '2025-01-22',
  'arrTime': '10:20:00',
  'arrDate': '2025-01-22',
  'minimumChangeDuration': 'PT0S'},
 {'name': 'Sandviken Mossvägen',
  'id': 'A=1@O=Sandviken Mossvägen@X=16791292@Y=60624241@U=1@L=740019988@',
  'extId': '740019988',
  'routeIdx': 6,
  'lon': 16.791292,
  'lat': 60.624241,
  'depTime': '10:21:00',
  'depDate': '2025-01-22',
  'arrTime': '10:21:00',
  'arrDate': '2025-01-22',
  'minimumChangeDuration': 'PT0S'},
 {'name': 'Valbo E16 (Gävl

In [67]:
for stop in stops:
    print(f"Hållplats: {stop.get('name')}")
    print(f"Ankomsttid: {stop.get('arrTime')}")
    print(f"Avgångstid: {stop.get('depTime')}")
    print(f"Koordinater: Lat {stop.get('lat')}, Lon {stop.get('lon')}")
    print("-" * 30)

Hållplats: Sandviken station
Ankomsttid: None
Avgångstid: 10:19:00
Koordinater: Lat 60.614344, Lon 16.777881
------------------------------
Hållplats: Sandviken Norra Porten
Ankomsttid: 10:20:00
Avgångstid: 10:20:00
Koordinater: Lat 60.619827, Lon 16.789584
------------------------------
Hållplats: Sandviken Mossvägen
Ankomsttid: 10:21:00
Avgångstid: 10:21:00
Koordinater: Lat 60.624241, Lon 16.791292
------------------------------
Hållplats: Valbo E16 (Gävle kn)
Ankomsttid: 10:29:00
Avgångstid: 10:29:00
Koordinater: Lat 60.631055, Lon 16.99026
------------------------------
Hållplats: Gustavsbro (Gävle kn)
Ankomsttid: 10:39:00
Avgångstid: 10:39:00
Koordinater: Lat 60.674455, Lon 17.108369
------------------------------
Hållplats: Karlsborg (Gävle kn)
Ankomsttid: 10:40:00
Avgångstid: 10:40:00
Koordinater: Lat 60.675327, Lon 17.114302
------------------------------
Hållplats: Gävle sjukhus Västra vägen
Ankomsttid: 10:40:00
Avgångstid: 10:40:00
Koordinater: Lat 60.675668, Lon 17.121979
--

In [68]:
legs = example_trip["LegList"]["Leg"]
for leg in legs:
    print(f"Transport: {leg.get('name')} ({leg.get('category')})")
    print(f"Från: {leg['Origin']['name']} ({leg['Origin']['time']})")
    print(f"Till: {leg['Destination']['name']} ({leg['Destination']['time']})")
    print("-" * 30)

Transport: Länstrafik - Buss 41 (BLT)
Från: Sandviken station (10:19:00)
Till: Gävle Centralstation (10:48:00)
------------------------------
Transport: Snabbtåg 569 (JST)
Från: Gävle Centralstation (11:14:00)
Till: Stockholm Centralstation (12:38:00)
------------------------------
Transport: Snabbtåg 435 (JST)
Från: Stockholm Centralstation (13:07:00)
Till: Göteborg Centralstation (16:41:00)
------------------------------


In [69]:
import folium

start_lat = float(legs[0]['Origin']['lat'])
start_lon = float(legs[0]['Origin']['lon'])

m = folium.Map(location=[start_lat, start_lon], zoom_start=7)

for leg in legs:
    folium.Marker(
        location=[float(leg['Origin']['lat']), float(leg['Origin']['lon'])],
        popup=f"Start: {leg['Origin']['name']} ({leg['Origin']['time']})",
        icon=folium.Icon(color="green")
    ).add_to(m)

    if 'Stops' in leg and leg['Stops']:
        stops = leg['Stops']['Stop']
        for stop in stops:
            folium.Marker(
                location=[float(stop['lat']), float(stop['lon'])],
                popup=f"Hållplats: {stop['name']} ({stop.get('depTime', 'N/A')})",
                icon=folium.Icon(color="blue", icon="info-sign")
            ).add_to(m)

    folium.Marker(
        location=[float(leg['Destination']['lat']), float(leg['Destination']['lon'])],
        popup=f"Slut: {leg['Destination']['name']} ({leg['Destination']['time']})",
        icon=folium.Icon(color="red")
    ).add_to(m)

m

## Total travel time and number of stops in between

In [70]:
import pandas as pd

time_data = []
for leg in legs:
    origin_time = pd.to_datetime(leg['Origin']['time'])
    destination_time = pd.to_datetime(leg['Destination']['time'])
    duration = destination_time - origin_time
    
    stops = leg.get('Stops', {}).get('Stop', [])
    num_stops_between = max(0, len(stops) - 2)
    
    time_data.append({
        "Transport": leg.get("name"),
        "Start": origin_time,
        "Slut": destination_time,
        "Total tid": duration,
        "Antal stopp emellan": num_stops_between
    })

df_time = pd.DataFrame(time_data)
print(df_time)

              Transport               Start                Slut  \
0  Länstrafik - Buss 41 2025-01-22 10:19:00 2025-01-22 10:48:00   
1          Snabbtåg 569 2025-01-22 11:14:00 2025-01-22 12:38:00   
2          Snabbtåg 435 2025-01-22 13:07:00 2025-01-22 16:41:00   

        Total tid  Antal stopp emellan  
0 0 days 00:29:00                    8  
1 0 days 01:24:00                    2  
2 0 days 03:34:00                    3  


## Transfer count

In [71]:
from backend.trips import TripPlanner

# 740000195=Sandviken station, 740000002=Göteborg centralstation
trip_planner = TripPlanner(origin_id=740000195, destination_id=740000002)

first_trip = trip_planner.trips[0]

legs = first_trip.get("LegList", {}).get("Leg", [])

num_changes = len(legs) - 1

print(f"Antal byten på denna resa: {num_changes}")

Antal byten på denna resa: 2


Slutsats: EDA för uppgift 4 - reseplaneraren
- karta med punkter för varje station mellan ursprung och slutdestination (Sandviken station - Göteborg Centralstation), total tid för resan, antal stopp emellan och antal byten.